# Preparação do Cadastro inicial de Pacientes de Avaí (SP) para a JCAFB-2025
Outubro de 2024

## Dados fornecidos pela Prefeitura de Avaí

Os dados foram fornecidos em arquivos (formato PDF) contendo os dados dos Pacientes de Crianças e Idosos, distribuídos em 7 Micro Áreas:

- Microarea 1 - Crianças.pdf
- Microarea 1 - Idosos.pdf
- Microarea 2 - Crianças.pdf
- Microarea 2 - Idosos.pdf
- Microarea 3 - Crianças.pdf
- Microarea 3 - Idosos.pdf
- Microarea 5 - Crianças.pdf
- Microarea 5 - Idosos.pdf
- Microarea 6 - Crianças.pdf
- Microarea 6 - Idosos.pdf
- Microarea 7 - Crianças.pdf
- Microarea 7 - Idosos.pdf


## Conversão do conteúdo dos aquivos fornecidos (.pdf) para Planilhas do Microsoft Excel (.xlsx)

Utilizando o método de copia e cola, os conteúdos de cada arquivo apresentado foram transferidos para Planilhas do Microsoft Excel, gerando um arquivo um arquivo .xlsx para cada arquivo .pdf apresentado:

- Microarea 1 - Criancas.xlsx
- Microarea 1 - Idosos.xlsx
- Microarea 2 - Criancas.xlsx
- Microarea 2 - Idosos.xlsx
- Microarea 3 - Criancas.xlsx
- Microarea 3 - Idosos.xlsx
- Microarea 5 - Criancas.xlsx
- Microarea 5 - Idosos.xlsx
- Microarea 6 - Criancas.xlsx
- Microarea 6 - Idosos.xlsx
- Microarea 7 - Criancas.xlsx
- Microarea 7 - Idosos.xlsx


No processo de colagem do conteúdo, uma única coluna foi utilizada ("**Texto PDF**) onde várias linhas (número variável) foram ocupadas.

## Processamento dos conteúdos dos arquivos .xlsx

In [9]:
import pandas as pd

### Funções de Suporte (1)

In [10]:
def string_end_clean(string):

    if string[-1] == ' ':
        string = string[:-1]
    if string[-1] == '.':
        string = string[:-1]
    if string[-1] == '-':
        string = string[:-1]
    if string[-1] == ' ':
        string = string[:-1]

    return string


In [11]:
def string_reformat(string):

    string = string.title()

    string = string.replace(" De ", " de ")
    string = string.replace(" Da ", " da ")
    string = string.replace(" Do ", " do ")
    string = string.replace(" Dos ", "dos ")

    return string


In [12]:
def cadastro_file_process(xls_in_path, sheet):

    xls_in = pd.ExcelFile(xls_in_path)
    table = pd.read_excel(xls_in, sheet_name = sheet)

    # Novas colunas inseridas no data frame
    # table.insert(0, 'Texto PDF', None)
    # Coluna contendo os dados originais provenientes do arquivo .pdf.
    table.insert(1, 'Flag', None)
    table.insert(2, 'Nome', None)
    table.insert(3, 'ID', None)
    table.insert(4, 'Nascimento', None)
    table.insert(5, 'Sexo', None)
    table.insert(6, 'Logradouro', None)
    table.insert(7, 'Numero', None)
    table.insert(8, 'Complemento', None)
    table.insert(9, 'Bairro', None)
    table.insert(10, 'Telefone', None)

    # Marcação da última linha das informações de cada Paciente.
    # Flag == '--'
    table.loc[(table['Texto PDF']=='CDS') |
              (table['Texto PDF']=='PEC'), 'Flag'] = '--'

    # Marcação da primeira linha das informações de cada Paciente.
    # Flag == '**'
    # Esta marcação indica a linha onde o nome do Paciente está armazenado.
    for i, row in table.iterrows():
        if i == 0:
            table['Flag'].values[i] = '**'
        if row['Flag'] == '--':
            try:
                table['Flag'].values[i + 1] = '**'
            except:
                pass

    # Marcação a linha onde está armazenado o número do CPF do Paciente.
    # Flag == 'cpf'
    # Caso esta não seja a segunda linha das informações do Paciente, a segunda linha é marcada com o
    # Flag == '*c' indicando que a segunda linha contêm a continuação do nome do Paciente.
    for i, row in table.iterrows():
        if row['Flag'] == '**':
            if table['Texto PDF'].values[i + 1][0:4] == 'CPF:':
                table['Flag'].values[i + 1] = 'cpf'
            else:
                if table['Texto PDF'].values[i + 2][0:4] == 'CPF:':
                    table['Flag'].values[i + 2] = 'cpf'
                    table['Flag'].values[i + 1] = '*c'

    # Marcação a linha onde está armazenado o número do CNS (Cartão Nacional de Saude) do Paciente.
    # Flag == 'cns'
    # Caso esta não seja a segunda linha das informações do Paciente, a segunda linha é marcada com o
    # Flag == '*c' indicando que a segunda linha contêm a continuação do nome do Paciente.
    for i, row in table.iterrows():
        if row['Flag'] == '**':
            if table['Texto PDF'].values[i + 1][0:4] == 'CNS:':
                table['Flag'].values[i + 1] = 'cns'
            else:
                if table['Texto PDF'].values[i + 2][0:4] == 'CNS:':
                    table['Flag'].values[i + 2] = 'cns'
                    table['Flag'].values[i + 1] = '*c'

    # Obtenção do Nome do Paciente
    for i, row in table.iterrows():
        if row['Flag'] == '**':
            if table['Flag'].values[i + 1] == '*c':
                name = row['Texto PDF'] + ' ' + table['Texto PDF'].values[i + 1]
            else:
                name = row['Texto PDF']
            name = string_reformat(name)
            table['Nome'].values[i] = name

    # Obtenção do número do CPF do Paciente
    for i, row in table.iterrows():
        if row['Flag'] == 'cpf':
            id = row['Texto PDF'][4:19]
            table['ID'].values[i] = id

    # Obtenção do número do CNS do Paciente
    for i, row in table.iterrows():
        if row['Flag'] == 'cns':
            id = row['Texto PDF'][4:20]
            table['ID'].values[i] = id

    # Obtenção do Gênero (Sexo) e da Data de Nascimento do Paciente
    for i, row in table.iterrows():
        text = str(row['Texto PDF'])
        sex = False
        if text.find("Feminino") != -1:
            sex = 'Feminino'
        if text.find("Masculino") != -1:
            sex = 'Masculino'
        if sex:
            table['Sexo'].values[i] = sex
            index = text.index('/')
            if index > 0:
                 table['Nascimento'].values[i] = text[index - 2:index + 8]

    # Obtenção do Número do Telefone do Paciente
    for i, row in table.iterrows():
        tel = False
        text = str(row['Texto PDF'])
        index = text.find("(1")
        if index != -1:
            tel = text[index:index + 15]
        if tel:
            table['Telefone'].values[i] = tel

    # Obtenção das informações completas do Endereço do Paciente

    for i, row in table.iterrows():
        logradouro = False
        numero = False
        complemento = False
        bairro = False
        text = str(row['Texto PDF'])

        # Obtenção do texto contendo todas as possíveis informações do Endereço do Paciente

        index = text.find("Rua") 
        if index != -1:
            logradouro = text[index:]
            if table['Flag'].values[i + 1] != '--':
                logradouro = logradouro + ' ' + table['Texto PDF'].values[i + 1]

        index = text.find("Avenida")
        if index != -1:
            logradouro = text[index:]
            if table['Flag'].values[i + 1] != '--':
                logradouro = logradouro + ' ' + table['Texto PDF'].values[i + 1]

        index = text.find("Travessa")
        if index != -1:
            logradouro = text[index:]
            if table['Flag'].values[i + 1] != '--':
                logradouro = logradouro + ' ' + table['Texto PDF'].values[i + 1]

        index = text.find("Praça")
        if index != -1:
            logradouro = text[index:]
            if table['Flag'].values[i + 1] != '--':
                logradouro = logradouro + ' ' + table['Texto PDF'].values[i + 1]

        index = text.find("Viela")
        if index != -1:
            logradouro = text[index:]
            if table['Flag'].values[i + 1] != '--':
                logradouro = logradouro + ' ' + table['Texto PDF'].values[i + 1]

        # Segmentação das informações de Endereço do Paciente

        if logradouro:

            # Exclusão das informações da Cidade (Avaí) e outras informações, caso tenham sido coletadas
            index = logradouro.find(', Avaí')
            if index != -1:
                logradouro = logradouro[:index]
            if logradouro[-1] == ',':
                logradouro = logradouro[:-1]

            # Identificação de Endereços localizados no Bairro "Centro".

            index = logradouro.find('CENTRO')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Centro'
            index = logradouro.find('Centro')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Centro'

            # Identificação de Endereços localizados no Bairro "Vila Oliveira".

            index = logradouro.find('VILA OLIVEIRA')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Vila Oliveira'
            index = logradouro.find('Vila Oliveira')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Vila Oliveira'

            # Identificação de Endereços localizados no Bairro "Núcleo Habitacional Eunice Andrade Moreira".
            index = logradouro.find('NÚCLEO HABITACIONAL EUNICE ANDRADE')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Núcleo Habitacional Eunice Andrade Moreira'
            index = logradouro.find('NUCLEO HABITACIONAL EUNICE ANDRADE')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Núcleo Habitacional Eunice Andrade Moreira'
            index = logradouro.find('NÚCLEO HABITACIONAL EUNICE')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Núcleo Habitacional Eunice Andrade Moreira'
            index = logradouro.find('Núcleo Habitacional Eunice Andrade Moreira')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Núcleo Habitacional Eunice Andrade Moreira'

            # Identificação de Endereços localizados no Bairro "Núcleo Habitacional Julio Rocha".

            index = logradouro.find('NÚCLEO HABITACIONAL JULIO ROCHA')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Núcleo Habitacional Julio Rocha'
            index = logradouro.find('NUCLEO HABITACIONAL JULIO ROCHA')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Núcleo Habitacional Julio Rocha'
            index = logradouro.find('Núcleo Habitacional Julio Rocha')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Nucleo Hab Julio Rocha'
            index = logradouro.find('Núcleo Habitacional Julio Rocha')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Núcleo Habitacional Julio Rocha'
            index = logradouro.find('JULIO ROCHA')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Núcleo Habitacional Julio Rocha'
            index = logradouro.find('Nucleo Hab Julio Rocha')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Núcleo Habitacional Julio Rocha'

            # Identificação de Endereços localizados no Bairro "Cohab".

            index = logradouro.find('COHAB')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Cohab'
            index = logradouro.find('COAHB')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Cohab'
            index = logradouro.find('Cohab')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Cohab'
            index = logradouro.find('Coab')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Cohab'

            # Identificação de Endereços localizados no Bairro "Conjunto Habitacional Manoel Rodrigues".

            index = logradouro.find('CONJUNTO HABITACIONAL MANOEL RODRIGUES')

            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Conjunto Habitacional Manoel Rodrigues'
            index = logradouro.find('Conjunto Habitacional Manoel Rodrigues')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Conjunto Habitacional Manoel Rodrigues'

            # Identificação de Endereços localizados no Bairro "Baixada".

            index = logradouro.find('BAIXADA')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Baixada'
            index = logradouro.find('Baixada')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Baixada'

            # Eliminação de caracteres espúrios do final do texto do Logradouro
            logradouro = string_end_clean(logradouro)

            # Obtenção das informações do Complemento da Residência dos Pacientes

            index = logradouro.find('CASA')
            if index != -1:
                complemento = logradouro[index:]
                logradouro = logradouro[:index]
            index = logradouro.find('Casa')
            if index != -1:
                complemento = logradouro[index:]
                logradouro = logradouro[:index]
            index = logradouro.find('FUNDOS')
            if index != -1:
                complemento = logradouro[index:]
                logradouro = logradouro[:index]
            index = logradouro.find('FUMDOS')
            if index != -1:
                complemento = logradouro[index:]
                logradouro = logradouro[:index]
            index = logradouro.find('Fundos')
            if index != -1:
                complemento = logradouro[index:]
                logradouro = logradouro[:index]
            index = logradouro.find('FRENTE')
            if index != -1:
                complemento = logradouro[index:]
                logradouro = logradouro[:index]
            index = logradouro.find('Frente')
            if index != -1:
                complemento = logradouro[index:]
                logradouro = logradouro[:index]
            index = logradouro.find('SOBRADO')
            if index != -1:
                complemento = logradouro[index:]
                logradouro = logradouro[:index]
            index = logradouro.find('Sobrado')
            if index != -1:
                complemento = logradouro[index:]
                logradouro = logradouro[:index]

            # Eliminação de caracteres espúrios do final do texto do Logradouro
            logradouro = string_end_clean(logradouro)

            # Obtenção das informações do Número da Residência dos Pacientes

            index = logradouro.find(', ')
            if index != -1:
                numero = logradouro[index + 2:]
                logradouro = logradouro[:index]

        # Armazenamento das informações do Endereço do Paciente

        if logradouro:
            logradouro = string_reformat(logradouro)
            table['Logradouro'].values[i] = logradouro
        if numero:
            table['Numero'].values[i] = numero
        if complemento:
            complemento = string_reformat(complemento)
            table['Complemento'].values[i] = complemento
        if bairro:
            table['Bairro'].values[i] = bairro

    return table

### Processamento dos arquivos de Cadastro

In [13]:
xls_out_path = 'data/avai_cadastro.xlsx'

In [14]:
xls_in_path = 'data/Microarea 1 - Criancas.xlsx'
sheet = 'MC1_Criancas'
table = cadastro_file_process(xls_in_path, sheet)
table.to_excel(xls_out_path, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 1 - Idosos.xlsx'
sheet = 'MC1_Idosos'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 2 - Criancas.xlsx'
sheet = 'MC2_Criancas'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 2 - Idosos.xlsx'
sheet = 'MC2_Idosos'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 3 - Criancas.xlsx'
sheet = 'MC3_Criancas'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 3 - Idosos.xlsx'
sheet = 'MC3_Idosos'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 5 - Criancas.xlsx'
sheet = 'MC5_Criancas'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 5 - Idosos.xlsx'
sheet = 'MC5_Idosos'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 6 - Criancas.xlsx'
sheet = 'MC6_Criancas'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 6 - Idosos.xlsx'
sheet = 'MC6_Idosos'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 7 - Criancas.xlsx'
sheet = 'MC7_Criancas'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 7 - Idosos.xlsx'
sheet = 'MC7_Idosos'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)


## Criação do Cadastro inicial de Pacientes da JCAFB-2025

### Funções de Suporte (2)

In [15]:
def jcafb_cadastro_file_process(xls_in_path, sheet, casdastro):

    xls_in = pd.ExcelFile(xls_in_path)
    table = pd.read_excel(xls_in, sheet_name = sheet, na_filter=False)

    nome = ''
    id_ = ''
    nascimento = ''
    sexo = ''
    logradouro = ''
    numero = ''
    complemento = ''
    bairro = ''
    telefone = ''
    fonte = sheet
    for i, row in table.iterrows():
        # print(row)
        if row['Flag'] == '**':
            nome = row['Nome']
        elif row['Flag'] == '--':
            new_row = {'Nome': nome,
                       'ID': id_,
                       'Nascimento': nascimento,
                       'Sexo': sexo,
                       'Logradouro': logradouro,
                       'Número': numero,
                       'Complemento': complemento,
                       'Bairro': bairro,
                       'Telefone': telefone,
                       'Fonte': fonte
                      }
            cadastro.loc[len(cadastro)] = new_row
        else:
            if row['ID'] != '':
                id_ = row['ID']
            if row['Nascimento'] != '':
                nascimento = row['Nascimento']
            if row['Sexo'] != '':
                sexo = row['Sexo']
            if row['Logradouro'] != '':
                logradouro = row['Logradouro']
            if row['Numero'] != '':
                numero = row['Numero']
            if row['Complemento'] != '':
                complemento = row['Complemento']
            if row['Bairro'] != '':
                bairro = row['Bairro']
            if row['Telefone'] != '':
                telefone = row['Telefone']


    return cadastro

### Criação do arquivo do Cadastro inicial de Pacientes da JCAFB-2025

In [16]:
xls_in_path = 'data/avai_cadastro.xlsx'
jcafb_cadastro_path = 'data/jcafb_cadastro.xlsx'

column_names=['Nome', 'ID', 'Nascimento', 'Sexo', 'Logradouro', 'Número', 'Complemento', 'Bairro', 'Telefone', 'Fonte']
cadastro=pd.DataFrame(columns=column_names)

sheet = 'MC1_Criancas'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, cadastro)

sheet = 'MC1_Idosos'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, cadastro)

sheet = 'MC2_Criancas'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, cadastro)

sheet = 'MC2_Idosos'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, cadastro)

sheet = 'MC3_Criancas'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, cadastro)

sheet = 'MC3_Idosos'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, cadastro)

sheet = 'MC5_Criancas'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, cadastro)

sheet = 'MC5_Idosos'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, cadastro)

sheet = 'MC6_Criancas'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, cadastro)

sheet = 'MC6_Idosos'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, cadastro)

sheet = 'MC7_Criancas'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, cadastro)

sheet = 'MC7_Idosos'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, cadastro)

cadastro.to_excel(jcafb_cadastro_path, sheet_name='Cadastro JCAFB-2025', index=True)

# print(cadastro)